# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! In this tutorial, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset

You can make a copy of this tutorial to play with it yourself.


# Install detectron2

In [13]:
# install dependencies
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-lp921k9d
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-lp921k9d
  Resolved https://github.com/facebookresearch/fvcore.git to commit e8d19df2ffdaf0a78c8d88d6a2522c36b0cacb07
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-t_mxw0kt
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-t_mxw0kt
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.6-py3-none-any.whl size=65324 sha256=15ab90c4074545f0c7c75475d163dfadef71a328c3261b4d771137078ed5d190
  Stored in directory: /tmp/pip-ephem-wheel-cache-syhr48bv/wheels/8f/cb/6a/3b7ac0e01781855ca3d1417ebf9e15e20d5b7fe37ab063aa50


'2.1.2+cu121'

In [14]:
#!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo
!pip install opencv-python

Obtaining file:///home/ec2-user/SageMaker/detectron2_repo
  Preparing metadata (setup.py) ... done
  Using cached pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
Using cached pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (426 kB)
  Attempting uninstall: fvcore
    Found existing installation: fvcore 0.1.6
    Uninstalling fvcore-0.1.6:
      Successfully uninstalled fvcore-0.1.6
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Uninstalling detectron2-0.6:
      Successfully uninstalled detectron2-0.6
  Running setup.py develop for detectron2


# Run a pre-trained detectron2 model

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [7]:
!pip install pytube
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import cv2
import random

# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from matplotlib import pyplot as plt

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)

from pytube import YouTube

def download_youtube_video(youtube_url, output_path='./'):
    yt = YouTube(youtube_url)
    video_stream = yt.streams.filter(file_extension='mp4').first()
    video_stream.download(output_path)
    return output_path

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.5 MB/s eta 0:00:00
[12/15 02:53:57 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [6]:
youtube_url = 'https://www.youtube.com/shorts/SCglass2C5I'
video_path = download_youtube_video(youtube_url)

NameError: name 'download_youtube_video' is not defined

In [200]:
import cv2
from detectron2.utils.visualizer import Visualizer
import random




cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)

def inference_on_video(video_path, output_video_path='output_video.mp4'):
    
    video_capture = cv2.VideoCapture(video_path)
        
    def get_random_color():
        return tuple(random.randint(0, 255) for _ in range(3))

    # Get video properties
    width = int(video_capture.get(3))
    height = int(video_capture.get(4))
    fps = int(video_capture.get(5))

    # Create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Get class names from COCO metadata
    class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes

    # Get class names from COCO metadata
    class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes

    # Store colors for each class
    class_colors = {class_name: get_random_color() for class_name in class_names}
    confidence_threshold = 0.55  
    



    counter = 1
    while True:  # Adjust the number of frames for testing
        # Read a frame from the video
        ret, frame = video_capture.read()
        if not ret:
            break

        # Perform inference on the frame
        outputs = predictor(frame)
        
        # Get instances and their corresponding colors
        instances = outputs["instances"]
        instance_boxes = instances.pred_boxes.tensor.cpu().numpy()
        instance_scores = instances.scores.cpu().numpy()
        instance_classes = instances.pred_classes.cpu().numpy()
        instance_masks = instances.pred_masks.cpu().numpy()


        
        for i in range(len(instance_scores)):
            box = instance_boxes[i]
            class_name = class_names[instance_classes[i]]
            score = instance_scores[i]
            mask = instance_masks[i]  # Get the mask for this instance

            # Check if the confidence is above the threshold
            print(score)
            print(class_name)
            print(box)
            if score >= confidence_threshold:
                # Convert to integers
                box = box.astype(int)

                # Get or generate a color for the class
                if class_name not in class_colors:
                    class_colors[class_name] = get_random_color()

                color = class_colors[class_name]

                # Draw bounding box on the frame
                frame = cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color, 2)

                # Add text annotation
                label = f"Class: {class_name}, Score: {score*100:.1f}%"
                frame = cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)

                # Draw segmentation mask
                alpha = 0.2
                for x in range(frame.shape[0]):
                    for y in range(frame.shape[1]):
                        # Check if any element in the segmentation mask is True
                        if mask[x, y]:
                            # If it's True and confidence is above the threshold, set the pixel to the mask color with transparency
                            frame[x, y] = (
                                alpha * color[0] + (1 - alpha) * frame[x, y][0],
                                alpha * color[1] + (1 - alpha) * frame[x, y][1],
                                alpha * color[2] + (1 - alpha) * frame[x, y][2],
                            )
                                

        # Write the frame with drawn bounding boxes to the output video
        out.write(frame)
        print(f"processing frame: {counter}")
        counter += 1
        # if counter == 32:
        #     break

    video_capture.release()
    out.release()
    cv2.destroyAllWindows()
    print("finished!")


[12/15 17:49:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [201]:
inference_on_video('./input_video_from_youtube.mp4')

0.8902468
car
[235.45714 322.7403  252.31728 332.58545]
0.7642816
truck
[259.99585 304.09836 335.53262 375.5096 ]
0.60702354
car
[  1.6720605 571.6894    357.34387   638.10443  ]
0.5422768
car
[260.34213 303.36768 334.01837 376.73108]
0.5177927
truck
[240.7361  312.53317 269.5625  332.045  ]
processing frame: 1
0.8845379
car
[232.6407 320.7269 249.2168 330.5851]
0.77107316
car
[  1.1989453 572.233     357.46732   638.28217  ]
0.74489576
truck
[253.76266 301.5111  331.19424 373.08502]
0.53034794
car
[252.88344 301.3054  332.23215 373.27686]
processing frame: 2
0.83914924
car
[232.43236 320.27097 249.29282 330.51086]
0.76867676
truck
[247.08899 300.57208 330.5349  373.52707]
0.7278926
car
[  2.1230567 572.3124    358.05762   637.7488   ]
0.51297027
car
[247.89526 300.96625 329.9608  373.7539 ]
processing frame: 3
0.92821825
car
[233.71298 323.342   250.49873 333.23434]
0.8365444
truck
[251.62796 301.9761  330.20352 376.28738]
0.74406606
car
[  1.8551513 573.4898    357.57117   638.19006 